In [168]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Load the Excel file
xls = pd.ExcelFile('https://github.com/saifrahmania/Data36118/raw/refs/heads/main/Assignment1/Data/ASCDataset/Australian%20Skills%20Classification%20-%20December%202023.xlsx')

# Dictionary to hold all DataFrames, one for each sheet
sheets_dict = {}

for sheet_name in xls.sheet_names:
    # Load each sheet into a DataFrame
    sheets_dict[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name)

xls.sheet_names

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['Index',
 'Glossary',
 'Occupation descriptions',
 'Core competency descriptions',
 'Specialist tasks hierarchy',
 'Tech tools heirarchy',
 'Core competencies',
 'Specialist tasks data',
 'Technology tools',
 'Appendix - tech tool examples',
 'Appendix - common tech tools']

In [169]:
for sheet_name in xls.sheet_names:
    print(f"Columns in sheet '{sheet_name}':")
    print(sheets_dict[sheet_name].columns.tolist())
    print("-" * 20)

Columns in sheet 'Index':
['Unnamed: 0', 'Unnamed: 1']
--------------------
Columns in sheet 'Glossary':
['Glossary of key terms', 'Unnamed: 1']
--------------------
Columns in sheet 'Occupation descriptions':
['Occupation Type', 'ANZSCO Code', 'Sub-Profile Code', 'ANZSCO Title', 'ANZSCO Description']
--------------------
Columns in sheet 'Core competency descriptions':
['Core Competency', 'Core Competency Description', 'Score', 'Proficiency Level', 'Anchor Value']
--------------------
Columns in sheet 'Specialist tasks hierarchy':
['Specialist Task', 'Specialist Cluster', 'Cluster Family', 'Skill Statement']
--------------------
Columns in sheet 'Tech tools heirarchy':
['Technology Tool Category', 'Technology Tool Category Description', 'Technology Tool', 'Technology Tool Description', 'Technology Tool Extended Description']
--------------------
Columns in sheet 'Core competencies':
['Occupation Type', 'ANZSCO Code', 'Sub-Profile Code', 'ANZSCO Title', 'Core Competency', 'Score', 'Pro

In [170]:
sheets_dict = {}

for sheet_name in xls.sheet_names:
    # Load each sheet into a DataFrame
    sheets_dict[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name)

# Accessing specific sheets and their data
index_df = sheets_dict['Index']
glossary_df = sheets_dict['Glossary']
occupation_descriptions_df = sheets_dict['Occupation descriptions']
core_competency_descriptions_df = sheets_dict['Core competency descriptions']
specialist_tasks_hierarchy_df = sheets_dict['Specialist tasks hierarchy']
tech_tools_hierarchy_df = sheets_dict['Tech tools heirarchy']
core_competencies_df = sheets_dict['Core competencies']
specialist_tasks_data_df = sheets_dict['Specialist tasks data']
technology_tools_df = sheets_dict['Technology tools']
appendix_tech_tool_examples_df = sheets_dict['Appendix - tech tool examples']
appendix_common_tech_tools_df = sheets_dict['Appendix - common tech tools']

# Now you can work with each DataFrame individually
# Example: Print the first 5 rows of the 'Occupation descriptions' sheet
index_df.head()

,Unnamed: 0,Unnamed: 1
0,Australian Skills Classification,NaN
1,Version 3.0 - current at December 2023 - updat...,NaN
2,Index,NaN
3,Glossary,Explanation of key terms used in this dataset.
4,Occupation descriptions,ANZSCO Occupation codes and descriptions.\nSub...


In [171]:
specialist_tasks_data_df.columns.tolist()

['Occupation Type',
 'ANZSCO Code',
 'Sub-Profile Code',
 'ANZSCO Title',
 'Specialist Task',
 '% of time spent on task',
 'Emerging/\nTrending Flag',
 'Specialist Cluster',
 ' % of time spent on cluster',
 'Cluster Family',
 '% of time spent on family',
 'Skills Statement']

In [172]:
print(core_competencies_df.columns.tolist())

['Occupation Type', 'ANZSCO Code', 'Sub-Profile Code', 'ANZSCO Title', 'Core Competency', 'Score', 'Proficiency Level', 'Anchor Value']


In [173]:
try:
  core_competencies_df = core_competencies_df.drop(columns=['Sub-Profile Code'])
except KeyError:
  print("Column 'Sub-Profile Code' not found in 'core_competencies_df'")

try:
  specialist_tasks_data_df = specialist_tasks_data_df.drop(columns=['Sub-Profile Code'])
except KeyError:
  print("Column 'Sub-Profile Code' not found in 'specialist_tasks_data_df'")

In [174]:
core_competencies_df.shape


(11030, 7)

In [175]:
import pandas as pd

# Assuming specialist_tasks_data_df and core_competencies_df are loaded DataFrames

# First, let's ensure there are no duplicates within core_competencies_df that could cause multiple matches
core_competencies_df = core_competencies_df.drop_duplicates(subset=['Occupation Type', 'ANZSCO Title'], keep='first')


# Perform a left join with core_competencies_df to append matching data
merged_df = pd.merge(specialist_tasks_data_df, core_competencies_df,
                     on=['Occupation Type', 'ANZSCO Title'],
                     how='left')

# Check and print the number of rows and structure to ensure it matches expectations
print("After merge, DataFrame size: ", merged_df.shape)
print(merged_df.head())

# Optionally, check for any rows that might still have missing data indicating no match was found
unmatched_indicator = merged_df.isna().any(axis=1)
print("Number of unmatched rows: ", unmatched_indicator.sum())

# This approach ensures we do not inadvertently increase the number of rows in specialist_tasks_data_df.



After merge, DataFrame size:  (30450, 16)
  Occupation Type  ANZSCO Code_x                             ANZSCO Title  \
0        ANZSCO 4           1111  Chief Executives and Managing Directors   
1        ANZSCO 4           1111  Chief Executives and Managing Directors   
2        ANZSCO 4           1111  Chief Executives and Managing Directors   
3        ANZSCO 4           1111  Chief Executives and Managing Directors   
4        ANZSCO 4           1111  Chief Executives and Managing Directors   

                                     Specialist Task  % of time spent on task  \
0  Direct or manage financial activities or opera...                   0.1302   
1     Direct department or organisational activities                   0.1117   
2  Direct sales, marketing or customer service ac...                   0.0808   
3  Communicate with others to arrange, coordinate...                   0.0665   
4  Analyse data to assess operational or project ...                   0.0651   

  Emergi

In [176]:
merged_df.shape

(30450, 16)

In [177]:
# prompt: name of all columns

merged_df.columns.tolist()


['Occupation Type',
 'ANZSCO Code_x',
 'ANZSCO Title',
 'Specialist Task',
 '% of time spent on task',
 'Emerging/\nTrending Flag',
 'Specialist Cluster',
 ' % of time spent on cluster',
 'Cluster Family',
 '% of time spent on family',
 'Skills Statement',
 'ANZSCO Code_y',
 'Core Competency',
 'Score',
 'Proficiency Level',
 'Anchor Value']

In [178]:
# prompt: drop 'Specialist Task',
#  '% of time spent on task',
#  'Emerging/\nTrending Flag', ' % of time spent on cluster','% of time spent on family','ANZSCO Code_y', from the merged_df

try:
  merged_df = merged_df.drop(columns=['% of time spent on task', 'Emerging/\nTrending Flag', ' % of time spent on cluster', '% of time spent on family', 'ANZSCO Code_y'])
except KeyError as e:
  print(f"Column not found: {e}")


In [179]:
merged_df.shape

(30450, 11)

In [180]:
# Count empty rows in each column of merged_df
empty_rows_per_column = merged_df.isnull().sum()

# Print the column names and the number of empty rows for each
for column, empty_count in empty_rows_per_column.items():
    print(f"Column '{column}': {empty_count} empty rows")

Column 'Occupation Type': 0 empty rows
Column 'ANZSCO Code_x': 0 empty rows
Column 'ANZSCO Title': 0 empty rows
Column 'Specialist Task': 0 empty rows
Column 'Specialist Cluster': 0 empty rows
Column 'Cluster Family': 0 empty rows
Column 'Skills Statement': 0 empty rows
Column 'Core Competency': 6568 empty rows
Column 'Score': 6568 empty rows
Column 'Proficiency Level': 6568 empty rows
Column 'Anchor Value': 6568 empty rows


In [181]:
# prompt: drop all the rows where 'Core Competency', 'Score' and 'Anchor Value' in 'merged_df'

# Drop rows where 'Core Competency', 'Score', and 'Anchor Value' are all null
merged_df = merged_df.dropna(subset=['Core Competency', 'Score', 'Anchor Value'], how='all')


In [182]:
merged_df.shape


(23882, 11)

In [183]:
merged_df.head()

,Occupation Type,ANZSCO Code_x,ANZSCO Title,Specialist Task,Specialist Cluster,Cluster Family,Skills Statement,Core Competency,Score,Proficiency Level,Anchor Value
0,ANZSCO 4,1111,Chief Executives and Managing Directors,Direct or manage financial activities or opera...,"Manage, monitor and undertake financial activi...",Business operations and financial activities,Direct and oversee the financial operations of...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...
1,ANZSCO 4,1111,Chief Executives and Managing Directors,Direct department or organisational activities,"Manage services, staff or activities",Business operations and financial activities,Direct and oversee the activities of a work un...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...
2,ANZSCO 4,1111,Chief Executives and Managing Directors,"Direct sales, marketing or customer service ac...","Manage services, staff or activities",Business operations and financial activities,"Direct and oversee the sales, marketing, or cu...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...
3,ANZSCO 4,1111,Chief Executives and Managing Directors,"Communicate with others to arrange, coordinate...",Communicate or collaborate with others,Communication and collaboration,"Coordinate with others in order to plan, organ...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...
4,ANZSCO 4,1111,Chief Executives and Managing Directors,Analyse data to assess operational or project ...,Use data to inform operational decisions,"Data, analytics, and databases",Analyse qualitative and quantitative data aris...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...


In [184]:
merged_df.columns.tolist()
merged_df.shape

(23882, 11)

In [185]:


try:
  merged_df = merged_df.rename(columns={'ANZSCO Code_x': 'ANZSCO Code'})
except KeyError as e:
  print(f"Column not found: {e}")

try:
    merged_df = merged_df.drop(columns=['Technology Tool_x', 'Technology Tool_y'])
except KeyError as e:
    print(f"Column not found: {e}")


Column not found: "['Technology Tool_x', 'Technology Tool_y'] not found in axis"


In [186]:
merged_df.columns.tolist()
occupation_descriptions_df.columns.tolist()

['Occupation Type',
 'ANZSCO Code',
 'Sub-Profile Code',
 'ANZSCO Title',
 'ANZSCO Description']

In [187]:
# prompt: match 'Occupation Type' and 'ANZSCO Title' from occupation_descriptions_df to merged_df. if matches inly add 'ANZSCO Description' from occupation_descriptions_df to merged_df

import pandas as pd
# Merge occupation descriptions based on 'Occupation Type' and 'ANZSCO Title'
merged_df = pd.merge(merged_df, occupation_descriptions_df[['Occupation Type', 'ANZSCO Title', 'ANZSCO Description']],
                     on=['Occupation Type', 'ANZSCO Title'],
                     how='left')
print(merged_df.shape)



(23882, 12)


In [188]:
# # Step 1: Sort both dataframes to ensure sequential matching
# merged_df = merged_df.sort_values(by=['Occupation Type', 'ANZSCO Title']).reset_index(drop=True)
# technology_tools_df = technology_tools_df.sort_values(by=['Occupation Type', 'ANZSCO Title']).reset_index(drop=True)

# # Step 2: Initialize indices for both dataframes
# tech_index = 0  # Tracks the position in technology_tools_df

# # Step 3: Iterate through merged_df and assign 'Technology Tool' sequentially
# for i in range(len(merged_df)):
#     while tech_index < len(technology_tools_df) and merged_df.loc[i, ['Occupation Type', 'ANZSCO Title']].equals(
#             technology_tools_df.loc[tech_index, ['Occupation Type', 'ANZSCO Title']]):
#         # Assign the Technology Tool value
#         merged_df.at[i, 'Technology Tool'] = technology_tools_df.loc[tech_index, 'Technology Tool']
#         tech_index += 1  # Move to the next technology tool

#     # Reset tech_index when end is reached
#     if tech_index >= len(technology_tools_df):
#         tech_index = 0

# # Step 4: Fill any remaining missing values with "No Specific Tools"
# merged_df['Technology Tool'].fillna("No Specific Tools", inplace=True)

# merged_df.head()

In [189]:
merged_df.shape

(23882, 12)

In [190]:
both_empty = merged_df[merged_df['Score'].isnull() & merged_df['Proficiency Level'].isnull()].shape[0]
print(f"Number of rows with both 'Score' and 'Proficiency Level' empty: {both_empty}")

# Count rows where 'Score' is empty/null/NaN but 'Proficiency Level' has a value
score_empty_proficiency_not = merged_df[merged_df['Score'].isnull() & merged_df['Proficiency Level'].notnull()].shape[0]
print(f"Number of rows with 'Score' empty but 'Proficiency Level' not empty: {score_empty_proficiency_not}")

# Count rows where 'Score' has a value but 'Proficiency Level' is empty/null/NaN
score_not_empty_proficiency_empty = merged_df[merged_df['Score'].notnull() & merged_df['Proficiency Level'].isnull()].shape[0]
print(f"Number of rows with 'Score' not empty but 'Proficiency Level' empty: {score_not_empty_proficiency_empty}")

# Count rows where either 'Score' or 'Proficiency Level' is empty/null/NaN
either_empty = merged_df[merged_df['Score'].isnull() | merged_df['Proficiency Level'].isnull()].shape[0]
print(f"Number of rows with either 'Score' or 'Proficiency Level' empty: {either_empty}")


Number of rows with both 'Score' and 'Proficiency Level' empty: 0
Number of rows with 'Score' empty but 'Proficiency Level' not empty: 0
Number of rows with 'Score' not empty but 'Proficiency Level' empty: 0
Number of rows with either 'Score' or 'Proficiency Level' empty: 0


In [192]:
# Calculate the mean of '% of time spent on task', '% of time spent on family', and 'Score', ignoring NaN values

mean_score = merged_df['Score'].mean()


merged_df['Score'].fillna(mean_score, inplace=True)

<ipython-input-192-02c996b9c362>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Score'].fillna(mean_score, inplace=True)


In [193]:
empty_rows_per_column = merged_df.isnull().sum()

# Print the column names and the number of empty rows for each
for column, empty_count in empty_rows_per_column.items():
    print(f"Column '{column}': {empty_count} empty rows")

Column 'Occupation Type': 0 empty rows
Column 'ANZSCO Code': 0 empty rows
Column 'ANZSCO Title': 0 empty rows
Column 'Specialist Task': 0 empty rows
Column 'Specialist Cluster': 0 empty rows
Column 'Cluster Family': 0 empty rows
Column 'Skills Statement': 0 empty rows
Column 'Core Competency': 0 empty rows
Column 'Score': 0 empty rows
Column 'Proficiency Level': 0 empty rows
Column 'Anchor Value': 0 empty rows
Column 'ANZSCO Description': 0 empty rows


In [194]:
# prompt: drop those rows where 'Core Competency','Anchor Value' and 'Core Competency' are null/NaN

# Drop rows where 'Core Competency', 'Anchor Value', and 'Core Competency' are null/NaN
merged_df = merged_df.dropna(subset=['Core Competency', 'Anchor Value'], how='any')

In [195]:
merged_df.shape

(23882, 12)

In [196]:
merged_df.head()

,Occupation Type,ANZSCO Code,ANZSCO Title,Specialist Task,Specialist Cluster,Cluster Family,Skills Statement,Core Competency,Score,Proficiency Level,Anchor Value,ANZSCO Description
0,ANZSCO 4,1111,Chief Executives and Managing Directors,Direct or manage financial activities or opera...,"Manage, monitor and undertake financial activi...",Business operations and financial activities,Direct and oversee the financial operations of...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...
1,ANZSCO 4,1111,Chief Executives and Managing Directors,Direct department or organisational activities,"Manage services, staff or activities",Business operations and financial activities,Direct and oversee the activities of a work un...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...
2,ANZSCO 4,1111,Chief Executives and Managing Directors,"Direct sales, marketing or customer service ac...","Manage services, staff or activities",Business operations and financial activities,"Direct and oversee the sales, marketing, or cu...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...
3,ANZSCO 4,1111,Chief Executives and Managing Directors,"Communicate with others to arrange, coordinate...",Communicate or collaborate with others,Communication and collaboration,"Coordinate with others in order to plan, organ...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...
4,ANZSCO 4,1111,Chief Executives and Managing Directors,Analyse data to assess operational or project ...,Use data to inform operational decisions,"Data, analytics, and databases",Analyse qualitative and quantitative data aris...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...


In [197]:
empty_rows_per_column = merged_df.isnull().sum()

# Print the column names and the number of empty rows for each
for column, empty_count in empty_rows_per_column.items():
    print(f"Column '{column}': {empty_count} empty rows")

Column 'Occupation Type': 0 empty rows
Column 'ANZSCO Code': 0 empty rows
Column 'ANZSCO Title': 0 empty rows
Column 'Specialist Task': 0 empty rows
Column 'Specialist Cluster': 0 empty rows
Column 'Cluster Family': 0 empty rows
Column 'Skills Statement': 0 empty rows
Column 'Core Competency': 0 empty rows
Column 'Score': 0 empty rows
Column 'Proficiency Level': 0 empty rows
Column 'Anchor Value': 0 empty rows
Column 'ANZSCO Description': 0 empty rows


In [198]:
# Drop the specified columns from technology_tools_df
technology_tools_df = technology_tools_df.drop(columns=['Emerging/Trending Flag', 'Sub-Profile Code'], errors='ignore')

# Display the updated DataFrame (optional)
technology_tools_df.head()

,Occupation Type,ANZSCO Code,ANZSCO Title,Technology Tool
0,ANZSCO 4,1111,Chief Executives and Managing Directors,Accounting and financial management systems
1,ANZSCO 4,1111,Chief Executives and Managing Directors,Audio/video conferencing software
2,ANZSCO 4,1111,Chief Executives and Managing Directors,Business intelligence and decision support sof...
3,ANZSCO 4,1111,Chief Executives and Managing Directors,Flow chart and diagram software
4,ANZSCO 4,1111,Chief Executives and Managing Directors,Human resources software


In [199]:
technology_tools_df.shape

(5761, 4)

In [200]:
technology_tools_aggregated = technology_tools_df.groupby(['Occupation Type', 'ANZSCO Title'])['Technology Tool'].apply(', '.join).reset_index()

# Merge final_df with this aggregated dataframe
merged_df = pd.merge(merged_df, technology_tools_aggregated, on=['Occupation Type', 'ANZSCO Title'], how='left')

# Now, merged_df will have an additional column 'Technology Tool' from technology_tools_df
# This column contains concatenated strings of tools, ensuring no increase in row count

# Display the head of the merged dataframe to verify
print(merged_df.head())

  Occupation Type  ANZSCO Code                             ANZSCO Title  \
0        ANZSCO 4         1111  Chief Executives and Managing Directors   
1        ANZSCO 4         1111  Chief Executives and Managing Directors   
2        ANZSCO 4         1111  Chief Executives and Managing Directors   
3        ANZSCO 4         1111  Chief Executives and Managing Directors   
4        ANZSCO 4         1111  Chief Executives and Managing Directors   

                                     Specialist Task  \
0  Direct or manage financial activities or opera...   
1     Direct department or organisational activities   
2  Direct sales, marketing or customer service ac...   
3  Communicate with others to arrange, coordinate...   
4  Analyse data to assess operational or project ...   

                                  Specialist Cluster  \
0  Manage, monitor and undertake financial activi...   
1               Manage services, staff or activities   
2               Manage services, staff or ac

In [201]:
for col in merged_df.columns:
    empty_count = merged_df[col].isnull().sum()
    print(f"Column '{col}': {empty_count} empty values")

Column 'Occupation Type': 0 empty values
Column 'ANZSCO Code': 0 empty values
Column 'ANZSCO Title': 0 empty values
Column 'Specialist Task': 0 empty values
Column 'Specialist Cluster': 0 empty values
Column 'Cluster Family': 0 empty values
Column 'Skills Statement': 0 empty values
Column 'Core Competency': 0 empty values
Column 'Score': 0 empty values
Column 'Proficiency Level': 0 empty values
Column 'Anchor Value': 0 empty values
Column 'ANZSCO Description': 0 empty values
Column 'Technology Tool': 2653 empty values


In [202]:
#merged_df['Technology Tool'].fillna('', inplace=True)

In [203]:
# Fill missing 'Technology Tool' values by iterating through 'technology_tools_df' again

# Identify rows with missing 'Technology Tool' values
missing_tech_tool_rows = merged_df[merged_df['Technology Tool'].isnull()]

# Iterate through missing rows and try to find a match in technology_tools_df
for index, row in missing_tech_tool_rows.iterrows():
    # Find matching entries in technology_tools_df
    matched_tools = technology_tools_df[
        (technology_tools_df['Occupation Type'] == row['Occupation Type']) &
        (technology_tools_df['ANZSCO Title'] == row['ANZSCO Title'])
    ]['Technology Tool'].dropna().unique()  # Drop NaN values and get unique tools

    # If matches are found, update 'Technology Tool' column in merged_df
    if len(matched_tools) > 0:
        merged_df.at[index, 'Technology Tool'] = ', '.join(matched_tools)

# Verify if missing values are reduced
empty_count_after_filling = merged_df['Technology Tool'].isnull().sum()
print(f"After filling missing values, empty 'Technology Tool' count: {empty_count_after_filling}")

After filling missing values, empty 'Technology Tool' count: 2653


In [204]:
# Step 1: Standardize column values (strip spaces, lowercase)
merged_df['Occupation Type'] = merged_df['Occupation Type'].str.strip().str.lower()
merged_df['ANZSCO Title'] = merged_df['ANZSCO Title'].str.strip().str.lower()
technology_tools_df['Occupation Type'] = technology_tools_df['Occupation Type'].str.strip().str.lower()
technology_tools_df['ANZSCO Title'] = technology_tools_df['ANZSCO Title'].str.strip().str.lower()

# Step 2: Re-attempt merging
technology_tools_aggregated = technology_tools_df.groupby(['Occupation Type', 'ANZSCO Title'])['Technology Tool']\
                                                 .apply(lambda x: ', '.join(x.dropna().unique())).reset_index()

# Step 3: Merge again
merged_df = pd.merge(merged_df, technology_tools_aggregated, on=['Occupation Type', 'ANZSCO Title'], how='left', suffixes=('', '_new'))

# Step 4: Fill missing Technology Tools from re-merged column
merged_df['Technology Tool'] = merged_df['Technology Tool'].fillna(merged_df['Technology Tool_new'])

# Drop helper column after merging
merged_df.drop(columns=['Technology Tool_new'], inplace=True)

# Step 5: Identify remaining missing values
missing_tech_tool_rows = merged_df[merged_df['Technology Tool'].isnull()]

# Step 6: Debug - Check if missing job titles exist in technology_tools_df
missing_titles = missing_tech_tool_rows[['Occupation Type', 'ANZSCO Title']].drop_duplicates()
unmatched_titles = missing_titles.merge(technology_tools_df[['Occupation Type', 'ANZSCO Title']],
                                        on=['Occupation Type', 'ANZSCO Title'],
                                        how='left', indicator=True)

# Print titles that don't exist in technology_tools_df
print("Job titles that are missing in technology_tools_df:")
print(unmatched_titles[unmatched_titles['_merge'] == 'left_only'])

# Step 7: Final Check - Print remaining empty values count
empty_count_after_filling = merged_df['Technology Tool'].isnull().sum()
print(f"\nAfter second attempt, empty 'Technology Tool' count: {empty_count_after_filling}")


Job titles that are missing in technology_tools_df:
    Occupation Type                                       ANZSCO Title  \
0          anzsco 4                         horticultural crop growers   
1          anzsco 4      amusement, fitness and sports centre managers   
2          anzsco 4                     marine transport professionals   
3          anzsco 4  other health diagnostic and promotion professi...   
4          anzsco 4     welfare, recreation and community arts workers   
..              ...                                                ...   
137        anzsco 6                          vending machine attendant   
138        anzsco 6                                 car park attendant   
139        anzsco 6                                crossing supervisor   
140        anzsco 6  electrical or telecommunications trades assistant   
141        anzsco 6                          ticket collector or usher   

        _merge  
0    left_only  
1    left_only  
2    lef

In [205]:
merged_df.head()

,Occupation Type,ANZSCO Code,ANZSCO Title,Specialist Task,Specialist Cluster,Cluster Family,Skills Statement,Core Competency,Score,Proficiency Level,Anchor Value,ANZSCO Description,Technology Tool
0,anzsco 4,1111,chief executives and managing directors,Direct or manage financial activities or opera...,"Manage, monitor and undertake financial activi...",Business operations and financial activities,Direct and oversee the financial operations of...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
1,anzsco 4,1111,chief executives and managing directors,Direct department or organisational activities,"Manage services, staff or activities",Business operations and financial activities,Direct and oversee the activities of a work un...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
2,anzsco 4,1111,chief executives and managing directors,"Direct sales, marketing or customer service ac...","Manage services, staff or activities",Business operations and financial activities,"Direct and oversee the sales, marketing, or cu...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
3,anzsco 4,1111,chief executives and managing directors,"Communicate with others to arrange, coordinate...",Communicate or collaborate with others,Communication and collaboration,"Coordinate with others in order to plan, organ...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
4,anzsco 4,1111,chief executives and managing directors,Analyse data to assess operational or project ...,Use data to inform operational decisions,"Data, analytics, and databases",Analyse qualitative and quantitative data aris...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."


In [206]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [207]:
from fuzzywuzzy import process

# Function to find closest matching job title
def find_closest_job(title, job_list):
    match, score = process.extractOne(title, job_list)
    return match if score > 80 else None  # Return only if similarity score > 80%

# Get unique job titles from technology_tools_df
existing_jobs = technology_tools_df['ANZSCO Title'].unique()

# Iterate through missing job titles and find closest matches
missing_tech_tool_rows = merged_df[merged_df['Technology Tool'].isnull()].copy()

for index, row in missing_tech_tool_rows.iterrows():
    closest_match = find_closest_job(row['ANZSCO Title'], existing_jobs)

    if closest_match:
        # Find corresponding technology tool for the closest matched job title
        matched_tools = technology_tools_df[technology_tools_df['ANZSCO Title'] == closest_match]['Technology Tool'].dropna().unique()

        if len(matched_tools) > 0:
            merged_df.at[index, 'Technology Tool'] = ', '.join(matched_tools)

# Final Check - Count remaining empty values
empty_count_final = merged_df['Technology Tool'].isnull().sum()
print(f"After fuzzy matching, empty 'Technology Tool' count: {empty_count_final}")

After fuzzy matching, empty 'Technology Tool' count: 713


In [208]:
# Assign "No Specific Tools" to remaining missing values
merged_df['Technology Tool'].fillna("No Specific Tools", inplace=True)

# Final check on missing values
empty_count_final = merged_df['Technology Tool'].isnull().sum()
print(f"After assigning default values, empty 'Technology Tool' count: {empty_count_final}")


After assigning default values, empty 'Technology Tool' count: 0


<ipython-input-208-6cf7402ffb4c>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Technology Tool'].fillna("No Specific Tools", inplace=True)


In [209]:
merged_df.head()

,Occupation Type,ANZSCO Code,ANZSCO Title,Specialist Task,Specialist Cluster,Cluster Family,Skills Statement,Core Competency,Score,Proficiency Level,Anchor Value,ANZSCO Description,Technology Tool
0,anzsco 4,1111,chief executives and managing directors,Direct or manage financial activities or opera...,"Manage, monitor and undertake financial activi...",Business operations and financial activities,Direct and oversee the financial operations of...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
1,anzsco 4,1111,chief executives and managing directors,Direct department or organisational activities,"Manage services, staff or activities",Business operations and financial activities,Direct and oversee the activities of a work un...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
2,anzsco 4,1111,chief executives and managing directors,"Direct sales, marketing or customer service ac...","Manage services, staff or activities",Business operations and financial activities,"Direct and oversee the sales, marketing, or cu...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
3,anzsco 4,1111,chief executives and managing directors,"Communicate with others to arrange, coordinate...",Communicate or collaborate with others,Communication and collaboration,"Coordinate with others in order to plan, organ...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
4,anzsco 4,1111,chief executives and managing directors,Analyse data to assess operational or project ...,Use data to inform operational decisions,"Data, analytics, and databases",Analyse qualitative and quantitative data aris...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."


In [210]:
# Drop the specified column from occupation_descriptions_df
occupation_descriptions_df = occupation_descriptions_df.drop(columns=['Sub-Profile Code'], errors='ignore')

# Display the updated DataFrame (optional)
occupation_descriptions_df.head()

,Occupation Type,ANZSCO Code,ANZSCO Title,ANZSCO Description
0,ANZSCO 4,1111,Chief Executives and Managing Directors,Chief Executives and Managing Directors determ...
1,ANZSCO 4,1112,General Managers,"General Managers plan, organise, direct, contr..."
2,ANZSCO 4,1211,Aquaculture Farmers,"Aquaculture Farmers plan, organise, control, c..."
3,ANZSCO 4,1213,Livestock Farmers,"Livestock Farmers plan, organise, control, coo..."
4,ANZSCO 4,1215,Broadacre Crop Growers,"Broadacre Crop Growers plan, organise, control..."


In [211]:
occupation_descriptions_df.shape

(1650, 4)

In [214]:
import pandas as pd

# Assuming merged_df is already loaded and contains the results from previous merging processes

# Perform the merge to add 'ANZSCO Description'
merged_df = pd.merge(merged_df, occupation_descriptions_df[['Occupation Type', 'ANZSCO Title', 'ANZSCO Description']],
                     on=['Occupation Type', 'ANZSCO Title'],
                     how='left')

# Now, merged_df will have an additional column 'ANZSCO Description' from occupation_descriptions_df
# This column contains descriptions based on the matched 'Occupation Type' and 'ANZSCO Title'

# Print the first few rows to verify the new structure and content


# Optionally, check the number of rows to ensure they haven't increased
print("Number of rows in the merged DataFrame:", len(merged_df))
merged_df.head()

Number of rows in the merged DataFrame: 23882


,Occupation Type,ANZSCO Code,ANZSCO Title,Specialist Task,Specialist Cluster,Cluster Family,Skills Statement,Core Competency,Score,Proficiency Level,Anchor Value,ANZSCO Description_x,Technology Tool,ANZSCO Description_y,ANZSCO Description
0,anzsco 4,1111,chief executives and managing directors,Direct or manage financial activities or opera...,"Manage, monitor and undertake financial activi...",Business operations and financial activities,Direct and oversee the financial operations of...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A...",NaN,NaN
1,anzsco 4,1111,chief executives and managing directors,Direct department or organisational activities,"Manage services, staff or activities",Business operations and financial activities,Direct and oversee the activities of a work un...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A...",NaN,NaN
2,anzsco 4,1111,chief executives and managing directors,"Direct sales, marketing or customer service ac...","Manage services, staff or activities",Business operations and financial activities,"Direct and oversee the sales, marketing, or cu...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A...",NaN,NaN
3,anzsco 4,1111,chief executives and managing directors,"Communicate with others to arrange, coordinate...",Communicate or collaborate with others,Communication and collaboration,"Coordinate with others in order to plan, organ...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A...",NaN,NaN
4,anzsco 4,1111,chief executives and managing directors,Analyse data to assess operational or project ...,Use data to inform operational decisions,"Data, analytics, and databases",Analyse qualitative and quantitative data aris...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A...",NaN,NaN


In [215]:
# prompt: drop 'ANZSCO Description_y'	'ANZSCO Description' from merged_df

try:
    merged_df = merged_df.drop(columns=['ANZSCO Description_y', 'ANZSCO Description'])
except KeyError as e:
    print(f"Column not found: {e}")


In [216]:
for col in merged_df.columns:
    empty_count = merged_df[col].isnull().sum()
    print(f"Column '{col}': {empty_count} empty values")

Column 'Occupation Type': 0 empty values
Column 'ANZSCO Code': 0 empty values
Column 'ANZSCO Title': 0 empty values
Column 'Specialist Task': 0 empty values
Column 'Specialist Cluster': 0 empty values
Column 'Cluster Family': 0 empty values
Column 'Skills Statement': 0 empty values
Column 'Core Competency': 0 empty values
Column 'Score': 0 empty values
Column 'Proficiency Level': 0 empty values
Column 'Anchor Value': 0 empty values
Column 'ANZSCO Description_x': 0 empty values
Column 'Technology Tool': 0 empty values


In [217]:
# prompt: Rename ''ANZSCO Description_x'' to ''ANZSCO Description'

try:
    merged_df = merged_df.rename(columns={'ANZSCO Description_x': 'ANZSCO Description'})
except KeyError as e:
    print(f"Column not found: {e}")


In [218]:
merged_df.head()

,Occupation Type,ANZSCO Code,ANZSCO Title,Specialist Task,Specialist Cluster,Cluster Family,Skills Statement,Core Competency,Score,Proficiency Level,Anchor Value,ANZSCO Description,Technology Tool
0,anzsco 4,1111,chief executives and managing directors,Direct or manage financial activities or opera...,"Manage, monitor and undertake financial activi...",Business operations and financial activities,Direct and oversee the financial operations of...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
1,anzsco 4,1111,chief executives and managing directors,Direct department or organisational activities,"Manage services, staff or activities",Business operations and financial activities,Direct and oversee the activities of a work un...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
2,anzsco 4,1111,chief executives and managing directors,"Direct sales, marketing or customer service ac...","Manage services, staff or activities",Business operations and financial activities,"Direct and oversee the sales, marketing, or cu...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
3,anzsco 4,1111,chief executives and managing directors,"Communicate with others to arrange, coordinate...",Communicate or collaborate with others,Communication and collaboration,"Coordinate with others in order to plan, organ...",Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."
4,anzsco 4,1111,chief executives and managing directors,Analyse data to assess operational or project ...,Use data to inform operational decisions,"Data, analytics, and databases",Analyse qualitative and quantitative data aris...,Digital engagement,6.0,Intermediate,Use software on a portable device to document ...,Chief Executives and Managing Directors determ...,"Accounting and financial management systems, A..."


In [219]:
merged_df.shape

(23882, 13)

In [220]:
tech_tools_hierarchy_df.head()

,Technology Tool Category,Technology Tool Category Description,Technology Tool,Technology Tool Description,Technology Tool Extended Description
0,Broadcasting and audio-visual production techn...,"Systems for audio, video, multimedia and news ...",Collaborative news production platforms,Software used to create and manage news conten...,Collaborative news production platforms encomp...
1,Broadcasting and audio-visual production techn...,"Systems for audio, video, multimedia and news ...",Music or sound editing software,"Software used to create, manipulate, and edit ...",Music and sound editing software incorporates ...
2,Broadcasting and audio-visual production techn...,"Systems for audio, video, multimedia and news ...",Sound and audio hardware,"Apparatus used to create, manipulate, mix and/...",Hardware and equipment used to generate electr...
3,Broadcasting and audio-visual production techn...,"Systems for audio, video, multimedia and news ...",Video creation and editing software,Software used to create and edit digital video...,Video creation and editing software enables th...
4,Communication technologies,"Data, voice and/or video communication platforms",Audio/video conferencing software,Software for collaboration using video or audi...,Audio/video conferencing software enables peop...


In [222]:
# prompt: find out the numbe NaN value for 'Technology Tool' column in merged_df

nan_count = merged_df['Technology Tool'].isnull().sum()
print(f"Number of NaN values in 'Technology Tool' column: {nan_count}")


Number of NaN values in 'Technology Tool' column: 0


In [221]:
tech_tools_hierarchy_df.columns.tolist()

['Technology Tool Category',
 'Technology Tool Category Description',
 'Technology Tool',
 'Technology Tool Description',
 'Technology Tool Extended Description']

In [243]:
# Step 1: Ensure both dataframes are sorted for sequential traversal
merged_df = merged_df.sort_values(by=['Technology Tool']).reset_index(drop=True)
tech_tools_hierarchy_df = tech_tools_hierarchy_df.sort_values(by=['Technology Tool']).reset_index(drop=True)

# Step 2: Initialize index for tracking tech_tools_hierarchy_df
tech_index = 0

# Step 3: Iterate through merged_df and match 'Technology Tool' with tech_tools_hierarchy_df
for i in range(len(merged_df)):
    while tech_index < len(tech_tools_hierarchy_df) and merged_df.loc[i, 'Technology Tool'] == tech_tools_hierarchy_df.loc[tech_index, 'Technology Tool']:
        # Assign values from tech_tools_hierarchy_df to merged_df
        merged_df.at[i, 'Technology Tool Description'] = tech_tools_hierarchy_df.loc[tech_index, 'Technology Tool Description']
        merged_df.at[i, 'Technology Tool Extended Description'] = tech_tools_hierarchy_df.loc[tech_index, 'Technology Tool Extended Description']
        merged_df.at[i, 'Technology Tool Category'] = tech_tools_hierarchy_df.loc[tech_index, 'Technology Tool Category']
        merged_df.at[i, 'Technology Tool Category Description'] = tech_tools_hierarchy_df.loc[tech_index, 'Technology Tool Category Description']

        tech_index += 1  # Move to the next entry in tech_tools_hierarchy_df

    # Reset tech_index when end is reached
    if tech_index >= len(tech_tools_hierarchy_df):
        tech_index = 0

# Step 4: Fill remaining missing values with "Not Available" to ensure no NaN values
columns_to_fill = ['Technology Tool Description', 'Technology Tool Extended Description',
                   'Technology Tool Category', 'Technology Tool Category Description']
merged_df[columns_to_fill] = merged_df[columns_to_fill].fillna("Not Available")

merged_df.shape


(23882, 17)

In [244]:
merged_df.columns.tolist()

['Occupation Type',
 'ANZSCO Code',
 'ANZSCO Title',
 'Specialist Task',
 'Specialist Cluster',
 'Cluster Family',
 'Skills Statement',
 'Core Competency',
 'Score',
 'Proficiency Level',
 'Anchor Value',
 'ANZSCO Description',
 'Technology Tool',
 'Technology Tool Category',
 'Technology Tool Category Description',
 'Technology Tool Description',
 'Technology Tool Extended Description']

In [239]:
merged_df.shape

(23882, 17)

In [245]:
merged_df.head()


,Occupation Type,ANZSCO Code,ANZSCO Title,Specialist Task,Specialist Cluster,Cluster Family,Skills Statement,Core Competency,Score,Proficiency Level,Anchor Value,ANZSCO Description,Technology Tool,Technology Tool Category,Technology Tool Category Description,Technology Tool Description,Technology Tool Extended Description
0,anzsco 6,394211,furniture finisher,"Grind materials, parts, or items",Operate production equipment and make products,Production processes and machinery,"Set up, adjust, and operate grinding tools or ...",Digital engagement,4.0,Intermediate,Recognise different ways to connect to the int...,"Applies finishes, such as stain, lacquer, pain...",Accounting and financial management systems,Financial management and service delivery plat...,Systems to undertake financial management and ...,"Software for managing accounts, inventory, and...",Accounting and financial management systems en...
1,anzsco 6,394211,furniture finisher,Select production input materials,Manage construction or production projects,Work activities preparation,Select appropriate production input materials ...,Digital engagement,4.0,Intermediate,Recognise different ways to connect to the int...,"Applies finishes, such as stain, lacquer, pain...",Accounting and financial management systems,Financial management and service delivery plat...,Systems to undertake financial management and ...,"Software for managing accounts, inventory, and...",Accounting and financial management systems en...
2,anzsco 6,394211,furniture finisher,Treat timber,Undertake woodworking or carpentry,Construction,Treat timber in order to protect it from deter...,Digital engagement,4.0,Intermediate,Recognise different ways to connect to the int...,"Applies finishes, such as stain, lacquer, pain...",Accounting and financial management systems,Financial management and service delivery plat...,Systems to undertake financial management and ...,"Software for managing accounts, inventory, and...",Accounting and financial management systems en...
3,anzsco 6,394211,furniture finisher,Shape surfaces or edges of wood work pieces,Undertake woodworking or carpentry,Construction,"Form specific shapes, patterns, textures or ot...",Digital engagement,4.0,Intermediate,Recognise different ways to connect to the int...,"Applies finishes, such as stain, lacquer, pain...",Accounting and financial management systems,Financial management and service delivery plat...,Systems to undertake financial management and ...,"Software for managing accounts, inventory, and...",Accounting and financial management systems en...
4,anzsco 6,394211,furniture finisher,Repair furniture or upholstery,Repair parts or components,Production processes and machinery,Return functionality or desired appearance to ...,Digital engagement,4.0,Intermediate,Recognise different ways to connect to the int...,"Applies finishes, such as stain, lacquer, pain...",Accounting and financial management systems,Financial management and service delivery plat...,Systems to undertake financial management and ...,"Software for managing accounts, inventory, and...",Accounting and financial management systems en...


In [246]:
# prompt: find out the ratio of missing values for Technology Tool Category	Technology Tool Category Description	Technology Tool Description	Technology Tool Extended Description columns

# Calculate the ratio of missing values for specific columns
columns_to_check = ['Technology Tool Category', 'Technology Tool Category Description', 'Technology Tool Description', 'Technology Tool Extended Description']
total_rows = len(merged_df)

for column in columns_to_check:
    missing_count = merged_df[column].isnull().sum()
    missing_ratio = missing_count / total_rows
    print(f"Ratio of missing values in '{column}': {missing_ratio:.4f}")


Ratio of missing values in 'Technology Tool Category': 0.0000
Ratio of missing values in 'Technology Tool Category Description': 0.0000
Ratio of missing values in 'Technology Tool Description': 0.0000
Ratio of missing values in 'Technology Tool Extended Description': 0.0000


In [247]:
# prompt: export dataframe as csv file

# Assuming 'merged_df' is the DataFrame you want to export
merged_df.to_csv('merged_data.csv', index=False)  # Set index=False to avoid saving row indices
